In [ ]:
%pip install -U requests "camelot-py[base]" 'PyPDF2<3.0' deep-translator thefuzz
!apt install ghostscript

  Using cached camelot_py-0.11.0-py3-none-any.whl.metadata (8.3 kB)
  Using cached pypdf-4.3.1-py3-none-any.whl.metadata (7.4 kB)
  Using cached ghostscript-0.7-py2.py3-none-any.whl.metadata (4.4 kB)
INFO: pip is looking at multiple versions of camelot-py[base] to determine which version is compatible with other requirements. This could take a while.
  Using cached camelot_py-0.10.1-py3-none-any.whl.metadata (8.3 kB)
  Using cached camelot_py-0.10.0-py3-none-any.whl.metadata (8.3 kB)
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
ghostscript is already the newest version (9.55.0~dfsg1-0ubuntu5.9).
0 upgraded, 0 newly installed, 0 to remove and 49 not upgraded.


In [ ]:
import requests
import camelot
import pandas as pd

def synevo():
  pdf_url = "https://clinicavoluntari.ro/wp-content/uploads/2023/04/Preturi-Analize-Synevo-Aprilie-2023.pdf"

  pdf = requests.get(pdf_url)

  with open("synevo.pdf", "wb") as f:
      f.write(pdf.content)

  tables = camelot.read_pdf('synevo.pdf', pages='all')

  df = pd.concat([table.df for table in tables[1:]])

  df = df.iloc[1:]
  df[2].drop_duplicates()

  return df[2].reset_index(drop=True)

s = synevo()
translated_s = list(s)

KeyboardInterrupt: 

In [ ]:
import requests
import camelot
import pandas as pd
import numpy as np

def reginamaria():
  pdf_url = "https://www.reginamaria.ro/upload/files/Anexa-Analize-Laborator.pdf"

  pdf = requests.get(pdf_url)

  with open("reginamaria.pdf", "wb") as f:
      f.write(pdf.content)

  tables = camelot.read_pdf('reginamaria.pdf', pages='all')

  df = pd.concat([table.df for table in tables])

  df = pd.concat([df[0], df[1], df[2], df[3]]).drop_duplicates()
  df.replace('', np.nan, inplace=True)
  df = df.dropna()

  return df.reset_index(drop=True)

rm = reginamaria()
translated_rm = list(rm)

In [ ]:
from deep_translator import GoogleTranslator

translated_s = GoogleTranslator(source='ro', target='en').translate_batch(list(s))

In [ ]:
translated_s_series = pd.Series(translated_s)
sdf = pd.concat([s, translated_s_series], axis=1)
sdf = sdf.rename(columns={2: 'Synevo', 0: 'English Synevo'})
sdf['Synevo'] = sdf['Synevo'].str.replace('\n', '')
sdf['English Synevo'] = sdf['English Synevo'].str.replace('\n', '')
sdf

In [ ]:
from deep_translator import GoogleTranslator

translated_rm = GoogleTranslator(source='ro', target='en').translate_batch(list(rm))

In [ ]:
translated_rm_series = pd.Series(translated_rm)
rmdf = pd.concat([rm, translated_rm_series], axis=1)
rmdf = rmdf.rename(columns={0: 'Regina Maria', 1: 'English Regina Maria'})
rmdf['Regina Maria'] = rmdf['Regina Maria'].str.replace('\n', '')
rmdf['English Regina Maria'] = rmdf['English Regina Maria'].str.replace('\n', '')
rmdf

In [ ]:
from thefuzz import fuzz

best_matches = {}
reverse_best_matches = {}

for rmline in rmdf.iloc:
  test = rmline['Regina Maria']
  eng = rmline['English Regina Maria']
  possible = None
  for test2 in sdf['Synevo']:
    t1 = test.replace('(', '').replace(')', '').replace(' ', '')
    t2 = test2.replace('(', '').replace(')', '').replace(' ', '')
    ratio = fuzz.partial_ratio(t1, t2)
    if ratio > 80:
      if possible is None or possible[0] < ratio:
        possible = (ratio, test, test2, eng)
  if possible is not None:
    best_matches[possible[2]] = (possible[0], possible[1], possible[3])
    reverse_best_matches[possible[1]] = (possible[0], possible[2], possible[3])

In [ ]:
final_df = pd.DataFrame([], columns=['Ratio', 'Synevo', 'Synevo English', 'Regina Maria', 'Regina Maria English'])

for i, row in sdf.iterrows():
  if row.iloc[0] in best_matches:
    final_df.loc[i] = [
        best_matches[row.iloc[0]][0],
        row.iloc[0],
        row.iloc[1],
        best_matches[row.iloc[0]][1],
        best_matches[row.iloc[0]][2],
    ]
  else:
    final_df.loc[i] = [None, row.iloc[0], row.iloc[1], None, None]

for i, row in rmdf.iterrows():
  if row.iloc[0] not in reverse_best_matches:
    final_df.loc[len(final_df)] = [None, None, None, row.iloc[0], row.iloc[1]]

final_df

In [ ]:
from google.colab import sheets

extras_sheet = sheets.InteractiveSheet(sheet_id='12rEWTwolEPUCIuWieY94XeK7vsOMZeia1KVkdWzBFwA', display=False, worksheet_name='Extras')
everything = pd.concat([final_df, extras_sheet.as_df()])
sheet = sheets.InteractiveSheet(df=everything, sheet_id='12rEWTwolEPUCIuWieY94XeK7vsOMZeia1KVkdWzBFwA', worksheet_name='DATA', display=False)

https://docs.google.com/spreadsheets/d/12rEWTwolEPUCIuWieY94XeK7vsOMZeia1KVkdWzBFwA#gid=1901647988
https://docs.google.com/spreadsheets/d/12rEWTwolEPUCIuWieY94XeK7vsOMZeia1KVkdWzBFwA#gid=0


/usr/local/lib/python3.10/dist-packages/google/colab/sheets.py:31: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  return frame.applymap(_clean_val).replace({np.nan: None})
